In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/chopin-midi-file/chopin.mid.mid
/kaggle/input/musicnet-dataset/musicnet.npz
/kaggle/input/musicnet-dataset/musicnet_metadata.csv
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2150_br25m3.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2138_br51n1m2.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2116_brahms_sonata_opus_120-1.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2118_brahms_sonata_opus_120-3.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2154_br18m1.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2157_br18m4.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2131_bra11-6.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2156_br18m3.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Brahms/2140_br51n1m4.mid
/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis

# How do we approach this problem ? 

well we know that gpt2 is a text generation model. In order to train it in this data , we need to convert the data into textual form. 

I can use both CSV And midi files to do this task , but i guess the second method will take a little longer. But if you think about the usecase of this model , providing a midi file is a much easier option

let's take a random CSV and see what do we have 

In [2]:
data = pd.read_csv('/kaggle/input/musicnet-dataset/musicnet/musicnet/train_labels/1727.csv')
data

,start_time,end_time,instrument,note,start_beat,end_beat,note_value
0,9182,90078,43,53,4.000,1.500,Dotted Quarter
1,9182,33758,42,65,4.000,0.500,Eighth
2,9182,62430,1,69,4.000,1.000,Quarter
3,9182,202206,44,41,4.000,3.500,Whole
4,9182,62430,1,81,4.000,1.000,Quarter
...,...,...,...,...,...,...,...
6575,19196894,19421150,44,29,365.000,3.000,Dotted Half
6576,19226590,19233758,1,60,365.375,0.125,Thirty Second
6577,19226590,19233758,1,48,365.375,0.125,Thirty Second
6578,19233758,19421150,1,65,365.500,2.500,Dotted Half


let's see if i can extract these out colums from midi.
for this task we will use 'pretty_midi'

In [3]:
pip install pretty_midi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 40.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592289 sha256=ee3e4260fea03dc75e030219e857fffee984c5b428675a727c2ef55b49d157cc
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0

[notice] A new release of pip is available: 23.0.1 -> 24.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pretty_midi

In [5]:
midi_data = pretty_midi.PrettyMIDI('/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven/2313_qt15_1.mid')

/usr/local/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [6]:
data = []

# Iterate over all instruments
for instrument in midi_data.instruments:
    for note in instrument.notes:
        start_time = note.start
        end_time = note.end
        instrument_name = instrument.name if instrument.name else 'Unnamed'
        note_name = pretty_midi.note_number_to_name(note.pitch)
        start_beat = midi_data.get_beats(start_time)
        end_beat = midi_data.get_beats(end_time)
        note_value = note.pitch
        
        data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, note_value])

In [7]:
# Create a DataFrame
columns = ['start_time', 'end_time', 'instrument', 'note', 'start_beat', 'end_beat', 'note_value']
df = pd.DataFrame(data, columns=columns)

# Save the DataFrame to a CSV file (optional)
df.to_csv('midi_data.csv', index=False)

print(df.head())

   start_time   end_time instrument note  \
0         0.0   0.587500    Violin1   A4   
1         9.6  11.987500    Violin1  D#4   
2        12.0  14.387500    Violin1   E4   
3        14.4  16.787500    Violin1   C5   
4        16.8  19.452777    Violin1   B4   

                                          start_beat  \
0  [0.0, 1.2000000000000002, 2.4000000000000004, ...   
1  [9.600000000000001, 10.8, 12.0, 13.2, 14.39999...   
2  [12.0, 13.2, 14.399999999999999, 15.5999999999...   
3  [14.4, 15.600000000000001, 16.8, 18.133333, 19...   
4  [16.8, 18.133333, 19.466666, 20.666666, 21.866...   

                                            end_beat  note_value  
0  [0.5875, 1.7875, 2.9875000000000003, 4.1875, 5...          69  
1  [11.9875, 13.1875, 14.3875, 15.587499999999999...          63  
2  [14.387500000000001, 15.587500000000002, 16.78...          64  
3  [16.7875, 18.119444114583334, 19.4527771145833...          72  
4  [19.452777114583334, 20.654166, 21.854166, 23....          7

 **Alright , it works, But i have noticed some problem.**
> * If i just train it on the CSV , we cannot extract some more important details like the duration and the velocity of the note. 
> * Now to make it perfect , we are gonna need those columns. 

# So ladies and gentleman ...
> We are going with midi to train the model

In [8]:
entire_data = []
def midi_data_extraction(file_path):
    # Check if the path is a directory
    if os.path.isdir(file_path):
        # Loop through all files in the directory
        for file_name in os.listdir(file_path):
            if file_name.endswith('.mid'):
                # Construct the full file path
                full_path = os.path.join(file_path, file_name)
                # Load the MIDI file
                midi_data = pretty_midi.PrettyMIDI(full_path)
                
                # Extract data for each note
                for instrument in midi_data.instruments:
                    for note in instrument.notes:
                        start_time = note.start
                        end_time = note.end
                        instrument_name = instrument.name if instrument.name else 'Unnamed'
                        note_name = pretty_midi.note_number_to_name(note.pitch)
                        start_beat = midi_data.get_beats(start_time)
                        end_beat = midi_data.get_beats(end_time)
                        note_value = note.pitch
                        duration = end_time - start_time  # Duration of the note
                        velocity = note.velocity  # Velocity of the note

                        # Add the data to the list
                        entire_data.append([start_time, end_time, instrument_name, note_name, start_beat, end_beat, note_value, duration, velocity])


# Lets try just beethoven for now

In [9]:
file_name = '/kaggle/input/musicnet-dataset/musicnet_midis/musicnet_midis/Beethoven'
midi_data_extraction(file_name)

/usr/local/lib/python3.10/site-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [10]:
columns = ['start_time', 'end_time', 'instrument', 'note', 'start_beat', 'end_beat', 'note_value', 'duration', 'velocity']

# Create a DataFrame with the collected data
df = pd.DataFrame(entire_data, columns=columns)

# Save the DataFrame to a CSV file (optional)
df.to_csv('entire_midi_data.csv', index=False)

# Print the first few rows of the DataFrame
print(df.head())

   start_time  end_time instrument note  \
0    1.444443  1.553239      Right  A#5   
1    1.555554  1.775461      Right  A#5   
2    1.777776  1.886572      Right  G#5   
3    1.888887  2.108794      Right   F5   
4    2.111109  2.219905      Right   D5   

                                          start_beat  \
0  [1.444443, 1.6666649999999998, 1.8888869999999...   
1  [1.555554, 1.777776, 1.999998, 2.22222, 2.4444...   
2  [1.777776, 1.999998, 2.22222, 2.444442, 2.6666...   
3  [1.888887, 2.111109, 2.333331, 2.5555529999999...   
4  [2.111109, 2.333331, 2.5555529999999997, 2.777...   

                                            end_beat  note_value  duration  \
0  [1.5532391875, 1.7754611875, 1.997683187499999...          82  0.108796   
1  [1.7754611875, 1.9976831874999998, 2.219905187...          82  0.219907   
2  [1.8865721875, 2.1087941875, 2.3310161875, 2.5...          80  0.108796   
3  [2.1087941875, 2.3310161875, 2.5532381875, 2.7...          77  0.219907   
4  [2.21990518

We have the csv , lets convert it to text for gpt2 model 

In [11]:
with open('midi_text_data.txt', 'w') as file:
    file.write('<|startoftext|>\n')
    for index, row in df.iterrows():
        text_line = f"start_time: {row['start_time']}, end_time: {row['end_time']}, instrument: {row['instrument']}, note: {row['note']}, start_beat: {row['start_beat']}, end_beat: {row['end_beat']}, note_value: {row['note_value']}, duration: {row['duration']}, velocity: {row['velocity']}\n"
        file.write(text_line)
    file.write('<|endoftext|>\n')


In [12]:
import os

# List files in the working directory
os.listdir('/kaggle/working')


['midi_data.csv',
 'midi_text_data.txt',
 '.virtual_documents',
 'entire_midi_data.csv']

#  Model

In [4]:
import tensorflow as tf
from transformers import GPT2Tokenizer
from tqdm import tqdm

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def load_tf_dataset(file_path, tokenizer, block_size=128, chunk_size=1000000):
    def chunk_generator():
        with open(file_path, 'r', encoding='utf-8') as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                yield chunk

    def process_chunk(chunk):
        tokens = tokenizer.tokenize(chunk)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        for i in range(0, len(ids) - block_size + 1, block_size):
            yield {
                "input_ids": ids[i:i+block_size],
                "attention_mask": [1] * block_size
            }

    def gen():
        for chunk in tqdm(chunk_generator(), desc="Processing chunks"):
            yield from process_chunk(chunk)

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
            }
        )
    )

# Load the dataset
train_dataset = load_tf_dataset('midi_text_data.txt', tokenizer)

In [2]:
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel, DataCollatorForLanguageModeling

# Load the tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

# Load the model
model = TFGPT2LMHeadModel.from_pretrained('gpt2')

# Data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

# Training parameters
batch_size = 4  # Adjust batch size as needed
num_epochs = 3  # Number of epochs
learning_rate = 5e-5

# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=optimizer, loss=loss_fn)

# Read the dataset from the text file
with open('/kaggle/working/midi_text_data.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Function to encode the dataset
def encode(text):
    inputs = tokenizer(text, return_tensors='tf', max_length=512, truncation=True, padding='max_length')
    inputs['labels'] = inputs['input_ids']
    return inputs

# Tokenize the dataset
encoded_lines = [encode(line) for line in lines]

# Convert to TensorFlow dataset
input_ids = [encoded_line['input_ids'] for encoded_line in encoded_lines]
labels = [encoded_line['labels'] for encoded_line in encoded_lines]

train_dataset = tf.data.Dataset.from_tensor_slices((input_ids, labels))
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)

# Train the model
history = model.fit(train_dataset, epochs=num_epochs)

# Save the model and tokenizer
model.save_pretrained('./gpt2-midi')
tokenizer.save_pretrained('./gpt2-midi')


E0000 00:00:1720957142.048606      13 common_lib.cc:798] Could not set metric server port: INVALID_ARGUMENT: Could not find SliceBuilder port 8471 in any of the 0 ports provided in `tpu_process_addresses`="local"
=== Source Location Trace: ===
learning/45eac/tfrc/runtime/common_lib.cc:479
D0714 11:39:02.056969043      13 config.cc:196]                        gRPC EXPERIMENT call_status_override_on_cancellation   OFF (default:OFF)
D0714 11:39:02.056984991      13 config.cc:196]                        gRPC EXPERIMENT call_v3                                OFF (default:OFF)
D0714 11:39:02.056988478      13 config.cc:196]                        gRPC EXPERIMENT canary_client_privacy                  ON  (default:ON)
D0714 11:39:02.056990961      13 config.cc:196]                        gRPC EXPERIMENT capture_base_context                   ON  (default:ON)
D0714 11:39:02.056993358      13 config.cc:196]                        gRPC EXPERIMENT client_idleness                        ON  (defau

FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/working/midi_text_data.txt'

# Testing our model

let's clearly define our functions 
> What we're gonna do now is create two functions 
> * one function will be called midi_to_text() , another will be called text_to_midi()
> * why we need these two functions you may ask. **When the user will give us his midi file , we will convert it into a text file , and then predict the next notes using our model , and again convert it into a midi file**

**We will later try and get audio out of the midi file in this notebook itself**

In [ ]:
def midi_to_text(midi_file):
    midi_data = pretty_midi.PrettyMIDI(midi_file)
    text_data = []
    
    for instrument in midi_data.instruments:
        for note in instrument.notes:
            start_time = note.start
            end_time = note.end
            instrument_name = instrument.name if instrument.name else 'Unnamed'
            note_name = pretty_midi.note_number_to_name(note.pitch)
            start_beat = midi_data.get_beats(start_time)[0] if midi_data.get_beats(start_time) else 0
            end_beat = midi_data.get_beats(end_time)[0] if midi_data.get_beats(end_time) else 0
            note_value = note.pitch
            duration = end_time - start_time  
            velocity = note.velocity  
            
            text_line = f"{start_time} {end_time} {instrument_name} {note_name} {start_beat} {end_beat} {note_value} {duration} {velocity}"
            text_data.append(text_line)
    
    return '\n'.join(text_data)

def text_to_midi(text_data):
    midi_data = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    
    for line in text_data.split('\n'):
        start_time, end_time, instrument_name, note_name, start_beat, end_beat, note_value, duration, velocity = line.split()
        note = pretty_midi.Note(
            velocity=int(velocity),
            pitch=int(note_value),
            start=float(start_time),
            end=float(end_time)
        )
        instrument.notes.append(note)
    
    midi_data.instruments.append(instrument)
    return midi_data

In [ ]:
def testing(midi_file):
    # Convert MIDI to text
    input_text = midi_to_text(midi_file)
    
    # Tokenize input text
    inputs = tokenizer(input_text, return_tensors='tf', truncation=True, padding=True)
    
    # Generate predictions using the model
    outputs = model.generate(**inputs, max_length=1024, num_return_sequences=1)
    predicted_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Convert predicted text back to MIDI
    predicted_midi = text_to_midi(predicted_text)
    
    # Save the predicted MIDI file
    predicted_midi.write('final_piece.mid')

Here i m taking chopin waltz in A minor as a midi file to compose over. I hope it works

In [ ]:
midi_file = '/kaggle/input/chopin-midi-file'
testing(midi_file)

Lets listen to the music now 

In [ ]:
!pip install pygame

import pygame

# Initialize the mixer
pygame.mixer.init()

# Load your MIDI file
midi_file = '/kaggle/working/final_piece.mid' 
pygame.mixer.music.load(midi_file)

# Play the MIDI file
pygame.mixer.music.play()

# Wait until the music finishes playing
while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)
